In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

## Data Loading

In [2]:
path_to_file = "shakespeare.txt"

In [3]:
text = open(path_to_file,'r').read()

In [4]:
print(text[4500:4800])

converted are
  From his low tract and look another way:
    So thou, thy self out-going in thy noon:
    Unlooked on diest unless thou get a son.


                     8
  Music to hear, why hear'st thou music sadly?
  Sweets with sweets war not, joy delights in joy:
  Why lov'st thou that which t


In [5]:
vocab = sorted(set(text))
print('Number of unique character in text is {}'.format(len(vocab)))

Number of unique character in text is 84


In [6]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}
char_to_ind['h']

63

In [7]:
ind_to_char = np.array(vocab)
ind_to_char[63]

'h'

In [10]:
encoded_text = np.array([char_to_ind[c] for c in text])
print('Total number of character in text is: {}'.format(encoded_text.shape[0]))

Total number of character in text is: 5445609


In [14]:
sample_text = text[:500]
print(sample_text)


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [13]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

## Creating Batches

In [15]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [16]:
len("""
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
""")

133

In [23]:
seq_len = 120
total_num_seq = len(text) // (seq_len+1)
print(f'Total number of sequence in text is: {total_num_seq}')

Total number of sequence in text is: 45005


In [29]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1,drop_remainder=True) 

In [30]:
def create_seq_targets(seq):
    
    input_txt = seq[:-1]
    target_txt = seq[1:]
    
    return input_txt,target_txt

In [32]:
dataset = sequences.map(create_seq_targets)

In [36]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [37]:
batch_size = 128
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [38]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

## Model

In [41]:
vocab_size = len(vocab)
embed_dim = 64
rnn_neurons = 1026

In [42]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [44]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    
    model = Sequential()
    
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True,
                  stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam',loss=sparse_cat_loss)
    
    return model

In [45]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size)

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [48]:
for input_example_batch, target_example_batch in dataset.take(1):
  
  example_bacth_predictions = model(input_example_batch)

In [49]:
sampled_indices = tf.random.categorical(example_bacth_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([80, 58, 81, 78, 11, 77, 58, 11, 78, 48, 51,  9, 34, 21, 60, 49, 36,
       66,  6, 10, 74, 80, 55, 41, 54, 55,  9, 48, 27, 69, 82, 66, 11, 31,
       13, 19, 50, 50, 60, 14,  8, 73, 53, 61, 51, 56, 31, 22, 58, 13, 44,
       26, 21,  6, 48, 48, 74,  0, 28, 17, 14, 30, 65,  1, 47, 79, 67, 13,
       32, 60, 43, 70, 29,  6, 35, 68, 51, 19, 38, 12, 10, 61, 56,  7, 36,
       38, 29, 50, 55, 10, 57, 73, 25, 50, 17, 40, 28, 32,  9,  3, 70,  8,
       61, 50, 27, 68, 73, 26, 20, 62, 25, 81, 33, 18, 38, 38, 60, 72, 12,
       67], dtype=int64)

In [50]:
ind_to_char[sampled_indices]

array(['y', 'c', 'z', 'w', '0', 'v', 'c', '0', 'w', 'W', 'Z', '-', 'I',
       ':', 'e', 'X', 'K', 'k', '(', '.', 's', 'y', '`', 'P', '_', '`',
       '-', 'W', 'B', 'n', '|', 'k', '0', 'F', '2', '8', 'Y', 'Y', 'e',
       '3', ',', 'r', ']', 'f', 'Z', 'a', 'F', ';', 'c', '2', 'S', 'A',
       ':', '(', 'W', 'W', 's', '\n', 'C', '6', '3', 'E', 'j', ' ', 'V',
       'x', 'l', '2', 'G', 'e', 'R', 'o', 'D', '(', 'J', 'm', 'Z', '8',
       'M', '1', '.', 'f', 'a', ')', 'K', 'M', 'D', 'Y', '`', '.', 'b',
       'r', '?', 'Y', '6', 'O', 'C', 'G', '-', '"', 'o', ',', 'f', 'Y',
       'B', 'm', 'r', 'A', '9', 'g', '?', 'z', 'H', '7', 'M', 'M', 'e',
       'q', '1', 'l'], dtype='<U1')

In [ ]:
epochs = 30
model.fit(dataset,epochs=epochs)

In [51]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1,None]))

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [54]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
    num_generate = gen_size
    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval,0)

    text_generated = []
    temperature = temp

    model.reset_states()
    
    for i in range(num_generate):
        
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)
        predictions = predictions / temperature

        predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id],0)
        text_generated.append(ind_to_char[predicted_id])

    return(start_seed+''.join(text_generated))

In [55]:
print(generate_text(model,'JULIET',gen_size=1000))

JULIETHER. Here is Belunia; in the mads of Rome-
    Two of foul blunteoner, who with HYMPERS
                    bearing a
    Sun on your bosom.
  DERBY. The King has carr'd the bookers.  [They converset, is dead Denmars, Saying his gracious Roman
    Divers of the men that warriors in my
    osturanch. I was aim in these men than fools are gallantly;
    About with death, I'll
    terr such peril and capable. I have not death to you, lady.
  Leon. Ay, madam, please your letters; no second, seeming gracious sove,
    and my father and myself am forsworn to both my mother.
  LORD. That you do know these fellows' sopey treason,
    Why he's a warrant from Tranio?
  PROVOST. I see, sir, you were so indeed nor then.
    But here's a question's nick; these are
    O victory!
  ALL YOURGER. Fair, thou liven, for she comes; and, honeyour doubt
    I gave thee cloudean wings. Come not prettily!
    Come, go to; But dead, for your hearts shall wash
    His very friends in the copture)
    We 